In [4]:

from src.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Recommenders.Neural.MultVAE_PyTorch_Recommender import MultVAERecommender_PyTorch
from src.Recommenders.MatrixFactorization.PureSVDRecommender import ScaledPureSVDRecommender
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

import numpy as np
import random
import os

os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))

SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


import pandas as pd
import scipy.sparse as sps

dataframe_items=pd.read_csv("Data/data_ICM_metadata.csv",low_memory=True)

icm=sps.csr_matrix((dataframe_items["data"].values,(dataframe_items["item_id"].values.astype(int),dataframe_items["feature_id"].values)))

urm_train = sps.load_npz('Data/URM_train.npz')
urm_test = sps.load_npz('Data/URM_test.npz')
urm_train_stacked=sps.vstack([urm_train,icm.T])

In [2]:
import sqlite3
database_path='optuna_hybrid_linear_multvae.db'
conn = sqlite3.connect(database_path)

In [5]:
slim_recommender=SLIMElasticNetRecommender(urm_train)
pure_svd_recommender=PureSVDItemRecommender(urm_train)
rp3beta_recommender=RP3betaRecommender(urm_train)
rp3beta_stacked_recommender=RP3betaRecommender(urm_train_stacked)
scaled_svd_recommender= ScaledPureSVDRecommender(urm_train)
multvae_recommender=MultVAERecommender_PyTorch(urm_train)

In [ ]:
folder_path='trained_models/train/'
slim_recommender.load_model(folder_path=folder_path,file_name='SLIMElasticNetRecommender.zip')
pure_svd_recommender.load_model(folder_path=folder_path,file_name='best_pure_svd.zip')
scaled_svd_recommender.load_model(folder_path=folder_path,file_name='best_scaled_svd.zip')
rp3beta_recommender.load_model(folder_path=folder_path,file_name='best_rp3beta.zip')
rp3beta_stacked_recommender.load_model(folder_path=folder_path,file_name='best_stacked_rp3beta.zip')
multvae_recommender.load_model(folder_path=folder_path,file_name='best_MultVAE.zip')

In [7]:
recommenders_list={}

recommenders_list['slim']=slim_recommender
recommenders_list['pure_svd']=pure_svd_recommender
recommenders_list['scaled_svd']=scaled_svd_recommender
recommenders_list['rp3beta']=rp3beta_recommender
recommenders_list['rp3beta_stacked']=rp3beta_stacked_recommender
recommenders_list['multvae']=multvae_recommender
recommenders=[
   recommenders_list['slim'],
recommenders_list['pure_svd'],
recommenders_list['scaled_svd'],
recommenders_list['rp3beta'],
recommenders_list['rp3beta_stacked'],
recommenders_list['multvae']
]

In [8]:
from src.Recommenders.Hybrids.LinearWeightedRecommender import LinearWeightedRecommender
def objective(trial):
    weights = [
        trial.suggest_float('slim', 1e-2, 0.8, log=True),
        trial.suggest_float('pure_svd', 1e-4, 0.2, log=True),
        trial.suggest_float('scaled_svd', 1e-4, 0.2, log=True),
        trial.suggest_float('rp3beta', 1e-3, 0.6, log=True),
        trial.suggest_float('rp3beta_stacked', 1e-3, 0.6, log=True),
        trial.suggest_float('multVAE', 1e-4, 0.2, log=True)

    ]

    evaluator=EvaluatorHoldout(urm_test, cutoff_list=[10])
    linear_combinator = LinearWeightedRecommender(urm_train, recommenders, weights)
    result_df, _ = evaluator.evaluateRecommender(linear_combinator)
    return result_df.loc[10, "MAP"]

In [ ]:
import optuna
from optuna.samplers import RandomSampler
study=optuna.create_study(direction='maximize',
    study_name='hyperparameters_hybrid_linear',
    storage=f'sqlite:///{database_path}',load_if_exists=True,sampler=RandomSampler(seed=SEED)
)

In [ ]:
study.optimize(objective, n_trials=400,n_jobs=4)